# N4 Bias + Denoise + Resample + Affine Updating + Padding (most samples)

In [ ]:
import os
import nibabel as nib
import numpy as np
import SimpleITK as sitk
from scipy.ndimage import zoom

# ---- Configuration ----
images_root = "../images"
masks_root = "../segmentations/automatic"
target_spacing = [1.0, 1.0, 1.0]
target_shape = (384, 384, 192)

# ---- Utility Functions ----
def update_affine_for_new_spacing(original_affine, new_spacing):
    new_affine = original_affine.copy()
    new_affine[:3, :3] = np.diag(new_spacing)
    return new_affine

def resample_image(img_np, orig_spacing, new_spacing, order=1):
    zoom_factors = np.array(orig_spacing) / np.array(new_spacing)
    print(f"[INFO] Resampling with zoom factors: {zoom_factors}, order={order}")
    return zoom(img_np, zoom=zoom_factors, order=order)

def n4_bias_correction(image_np):
    print("[STEP] Running N4 bias correction...")
    sitk_image = sitk.GetImageFromArray(image_np.astype(np.float32))
    mask_image = sitk.OtsuThreshold(sitk_image, 0, 1, 200)
    corrector = sitk.N4BiasFieldCorrectionImageFilter()
    corrected = corrector.Execute(sitk_image, mask_image)
    print("[DONE] N4 bias correction complete.")
    return sitk.GetArrayFromImage(corrected)

def denoise_image(image_np):
    print("[STEP] Running denoising...")
    sitk_image = sitk.GetImageFromArray(image_np.astype(np.float32))
    denoised = sitk.CurvatureFlow(image1=sitk_image, timeStep=0.125, numberOfIterations=5)
    print("[DONE] Denoising complete.")
    return sitk.GetArrayFromImage(denoised)

def pad_to_shape(volume, target_shape):
    print(f"[INFO] Padding shape {volume.shape} → {target_shape}")
    pad_widths = []
    for dim_size, target in zip(volume.shape, target_shape):
        if dim_size > target:
            print(f"[WARNING] Skipping — dimension {dim_size} exceeds target {target}")
            return None
        pad_total = target - dim_size
        pad_before = pad_total // 2
        pad_after = pad_total - pad_before
        pad_widths.append((pad_before, pad_after))
    print(f"[INFO] Padding widths: {pad_widths}")
    return np.pad(volume, pad_widths, mode='constant', constant_values=0)

# ---- Main Loop ----
print("[INIT] Starting batch processing (patients 101–200)...")

all_patient_ids = sorted(os.listdir(images_root))
processed_count = 0

# Process only patients 101–200 (index 100 to 199)
for patient_id in all_patient_ids[100:200]:
    patient_folder = os.path.join(images_root, patient_id)
    if not os.path.isdir(patient_folder):
        continue

    for fname in os.listdir(patient_folder):
        if fname.startswith("DUKE_") and fname.endswith("0001.nii.gz"):
            input_path = os.path.join(patient_folder, fname)
            base_name = fname.replace(".nii.gz", "")
            output_image_path = os.path.join(patient_folder, f"{base_name}_n4_denoised_resampled_padded.nii.gz")

            mask_input_path = os.path.join(masks_root, f"{patient_id}.nii.gz")
            mask_output_path = os.path.join(masks_root, f"{patient_id}_processed.nii.gz")

            print(f"\n[PROCESSING] {input_path}")

            try:
                # Load and preprocess image
                img_nib = nib.load(input_path)
                img_np = img_nib.get_fdata()
                orig_affine = img_nib.affine
                orig_spacing = img_nib.header.get_zooms()[:3]
                print(f"[INFO] Image shape: {img_np.shape}, spacing: {orig_spacing}")

                img_n4 = n4_bias_correction(img_np)
                img_denoised = denoise_image(img_n4)
                img_resampled = resample_image(img_denoised, orig_spacing, target_spacing)

                padded_img = pad_to_shape(img_resampled, target_shape)
                if padded_img is None:
                    print("[SKIPPED] Image too large to pad.")
                    break

                new_affine = update_affine_for_new_spacing(orig_affine, target_spacing)
                nib.save(nib.Nifti1Image(padded_img, new_affine), output_image_path)
                print(f"[✅ SAVED] Image: {output_image_path}")

                # --- Process mask if it exists ---
                if os.path.exists(mask_input_path):
                    print(f"[STEP] Processing mask: {mask_input_path}")
                    mask_nib = nib.load(mask_input_path)
                    mask_np = mask_nib.get_fdata()
                    mask_spacing = mask_nib.header.get_zooms()[:3]

                    mask_resampled = resample_image(mask_np, mask_spacing, target_spacing, order=0)
                    padded_mask = pad_to_shape(mask_resampled, target_shape)
                    if padded_mask is None:
                        print("[SKIPPED] Mask too large to pad.")
                        break

                    nib.save(nib.Nifti1Image(padded_mask, new_affine), mask_output_path)
                    print(f"[✅ SAVED] Mask: {mask_output_path}")
                else:
                    print(f"[WARNING] Mask not found for {patient_id}, skipping mask.")

                processed_count += 1
                break  # Move to next patient after processing one image

            except Exception as e:
                print(f"[❌ ERROR] Failed processing {input_path}: {e}")
                break

print(f"\n[COMPLETE] Processed {processed_count} patients (101–200).")


[INIT] Starting batch processing...

[PROCESSING] images\DUKE_001\duke_001_0001.nii.gz
[INFO] Image shape: (448, 448, 160), spacing: (0.8035714, 0.8035714, 1.1)
[STEP] Running N4 bias correction...
[DONE] N4 bias correction complete.
[STEP] Running denoising...
[DONE] Denoising complete.
[INFO] Resampling with zoom factors: [0.8035714  0.8035714  1.10000002], order=1
[INFO] Padding shape (360, 360, 176) → (384, 384, 192)
[INFO] Padding widths: [(12, 12), (12, 12), (8, 8)]
[✅ SAVED] Image: images\DUKE_001\duke_001_0001_n4_denoised_resampled_padded.nii.gz
[STEP] Processing mask: segmentations/automatic\DUKE_001.nii.gz
[INFO] Resampling with zoom factors: [0.8035714  0.8035714  1.10000002], order=0
[INFO] Padding shape (360, 360, 176) → (384, 384, 192)
[INFO] Padding widths: [(12, 12), (12, 12), (8, 8)]
[✅ SAVED] Mask: segmentations/automatic\DUKE_001_processed.nii.gz

[PROCESSING] images\DUKE_002\duke_002_0001.nii.gz
[INFO] Image shape: (512, 512, 142), spacing: (0.5859, 0.5859, 1.300000

# N4 Bias + Denoise + Resample + Affine Updating + Isotropic Resizing + Padding (skipped samples)

In [ ]:
import os
import SimpleITK as sitk
import numpy as np
from scipy.ndimage import zoom

# Paths
images_root = "../images"
masks_root = "../segmentations/automatic"
target_shape = (384, 384, 192)
missing_patients = [
    "DUKE_012", "DUKE_028", "DUKE_046", "DUKE_051", "DUKE_069", "DUKE_071", "DUKE_082", "DUKE_086", "DUKE_090",
    "DUKE_097", "DUKE_107", "DUKE_116", "DUKE_117", "DUKE_141", "DUKE_168", "DUKE_178", "DUKE_183", "DUKE_185",
    "DUKE_228", "DUKE_236", "DUKE_240", "DUKE_250", "DUKE_253", "DUKE_285", "DUKE_290", "DUKE_298", "DUKE_307",
    "DUKE_321", "DUKE_346", "DUKE_350", "DUKE_360", "DUKE_368", "DUKE_385", "DUKE_406", "DUKE_407", "DUKE_412",
    "DUKE_420", "DUKE_438", "DUKE_444", "DUKE_465", "DUKE_470", "DUKE_474", "DUKE_482", "DUKE_525", "DUKE_527",
    "DUKE_535", "DUKE_543", "DUKE_562", "DUKE_567", "DUKE_610", "DUKE_612", "DUKE_615", "DUKE_623", "DUKE_633",
    "DUKE_645", "DUKE_660", "DUKE_662", "DUKE_666", "DUKE_687", "DUKE_688", "DUKE_697", "DUKE_717", "DUKE_724",
    "DUKE_751", "DUKE_754", "DUKE_774", "DUKE_775", "DUKE_776", "DUKE_778", "DUKE_780", "DUKE_789", "DUKE_792",
    "DUKE_816", "DUKE_817", "DUKE_873", "DUKE_874", "DUKE_886", "DUKE_916", "DUKE_917"
]

def save_image(image, path):
    sitk.WriteImage(image, path)

def sitk_to_numpy(image):
    return sitk.GetArrayFromImage(image), image.GetSpacing(), image.GetOrigin(), image.GetDirection()

def numpy_to_sitk(np_array, spacing, origin, direction):
    image = sitk.GetImageFromArray(np_array)
    image.SetSpacing(spacing)
    image.SetOrigin(origin)
    image.SetDirection(direction)
    return image

def resize_image_to_fit(np_image, target_shape):
    current_shape = np_image.shape
    scale = min(t / c for t, c in zip(target_shape, current_shape))
    zoom_factors = [scale] * len(current_shape)
    resized = zoom(np_image, zoom_factors, order=1)
    return resized

for patient_id in missing_patients:
    try:
        image_path = os.path.join(images_root, patient_id, f"{patient_id}_0001.nii.gz")
        mask_path = os.path.join(masks_root, f"{patient_id}_processed.nii.gz")

        if not os.path.exists(image_path) or not os.path.exists(mask_path):
            print(f"[SKIP] {patient_id}: Missing image or mask.")
            continue

        base_dir = os.path.join(images_root, patient_id)

        # Load image and mask
        image = sitk.ReadImage(image_path)
        mask = sitk.ReadImage(mask_path)

        # Step 1: N4 bias correction
        otsu_mask = sitk.OtsuThreshold(image, 0, 1, 200)
        corrector = sitk.N4BiasFieldCorrectionImageFilter()
        image_n4 = corrector.Execute(image, otsu_mask)
        save_image(image_n4, os.path.join(base_dir, f"{patient_id}_0001_n4.nii.gz"))

        # Step 2: Denoising
        image_denoised = sitk.CurvatureFlow(image1=image_n4, timeStep=0.125, numberOfIterations=5)
        save_image(image_denoised, os.path.join(base_dir, f"{patient_id}_0001_n4_denoised.nii.gz"))

        # Step 3: Resampling image and mask to isotropic 1mm
        def resample(img, interpolator):
            spacing = img.GetSpacing()
            size = img.GetSize()
            new_spacing = [1.0, 1.0, 1.0]
            new_size = [int(round(sz * spc / nsp)) for sz, spc, nsp in zip(size, spacing, new_spacing)]
            resampler = sitk.ResampleImageFilter()
            resampler.SetOutputSpacing(new_spacing)
            resampler.SetSize(new_size)
            resampler.SetOutputOrigin(img.GetOrigin())
            resampler.SetOutputDirection(img.GetDirection())
            resampler.SetInterpolator(interpolator)
            return resampler.Execute(img)

        image_resampled = resample(image_denoised, sitk.sitkLinear)
        mask_resampled = resample(mask, sitk.sitkNearestNeighbor)
        save_image(image_resampled, os.path.join(base_dir, f"{patient_id}_0001_n4_denoised_resampled.nii.gz"))
        save_image(mask_resampled, os.path.join(base_dir, f"{patient_id}_processed_resampled.nii.gz"))

        # Step 4: Resize to fit inside target shape
        def resize_sitk_image(img, order):
            np_img, spacing, origin, direction = sitk_to_numpy(img)
            resized_np = resize_image_to_fit(np_img, target_shape)
            return numpy_to_sitk(resized_np, [1.0, 1.0, 1.0], origin, direction)

        image_resized = resize_sitk_image(image_resampled, order=1)
        mask_resized = resize_sitk_image(mask_resampled, order=0)
        save_image(image_resized, os.path.join(base_dir, f"{patient_id}_0001_n4_denoised_resampled_resized.nii.gz"))
        save_image(mask_resized, os.path.join(base_dir, f"{patient_id}_processed_resampled_resized.nii.gz"))

        print(f"[DONE] Processed {patient_id}")

    except Exception as e:
        print(f"[ERROR] Failed on {patient_id}: {e}")


# Miscellaneous Code

## N4 Bias + Denoise + Resample + Affine Updating (1 sample)

In [27]:
import os
import nibabel as nib
import numpy as np
import SimpleITK as sitk
from scipy.ndimage import zoom

# ---- Configuration ----
input_path = "images/DUKE_313/duke_313_0001.nii.gz"
output_path = "images/DUKE_313/duke_313_n4_denoised_resampled_fixed.nii.gz"
target_spacing = [1.0, 1.0, 1.0]

# ---- Affine Update Function ----
def update_affine_for_new_spacing(original_affine, new_spacing):
    new_affine = original_affine.copy()
    new_affine[:3, :3] = np.diag(new_spacing)
    return new_affine

# ---- Resampling Function ----
def resample_image(img_np, orig_spacing, new_spacing, order=1):
    zoom_factors = np.array(orig_spacing) / np.array(new_spacing)
    return zoom(img_np, zoom=zoom_factors, order=order)

# ---- N4 Bias Correction ----
def n4_bias_correction(image_np):
    sitk_image = sitk.GetImageFromArray(image_np.astype(np.float32))
    mask_image = sitk.OtsuThreshold(sitk_image, 0, 1, 200)
    corrector = sitk.N4BiasFieldCorrectionImageFilter()
    corrected = corrector.Execute(sitk_image, mask_image)
    return sitk.GetArrayFromImage(corrected)

# ---- Denoising ----
def denoise_image(image_np):
    sitk_image = sitk.GetImageFromArray(image_np.astype(np.float32))
    denoised = sitk.CurvatureFlow(image1=sitk_image, timeStep=0.125, numberOfIterations=5)
    return sitk.GetArrayFromImage(denoised)

# ---- Load Original Image ----
img_nib = nib.load(input_path)
img_np = img_nib.get_fdata()
orig_affine = img_nib.affine
orig_spacing = img_nib.header.get_zooms()[:3]

# ---- Apply N4 Correction ----
print("[STEP] N4 bias correction...")
img_n4 = n4_bias_correction(img_np)

# ---- Apply Denoising ----
print("[STEP] Denoising...")
img_denoised = denoise_image(img_n4)

# ---- Apply Resampling ----
print("[STEP] Resampling...")
resampled_np = resample_image(img_denoised, orig_spacing, target_spacing)

# ---- Update Affine ----
new_affine = update_affine_for_new_spacing(orig_affine, target_spacing)

# ---- Save Result ----
resampled_nib = nib.Nifti1Image(resampled_np, affine=new_affine)
nib.save(resampled_nib, output_path)

print(f"[DONE] Saved resampled image with updated spacing to {output_path}")


[STEP] N4 bias correction...
[STEP] Denoising...
[STEP] Resampling...
[DONE] Saved resampled image with updated spacing to images/DUKE_313/duke_313_n4_denoised_resampled_fixed.nii.gz


In [23]:
img = nib.load(output_path)
print("Voxel size:", img.header.get_zooms()[:3])


Voxel size: (1.0, 1.0, 1.0)


## Resampling Size Test

In [3]:
import nibabel as nib

img = nib.load("images/DUKE_001/duke_001_0001.nii.gz")
voxel_size = img.header.get_zooms()[:3]
print("Original voxel size (mm):", voxel_size)


Original voxel size (mm): (0.8035714, 0.8035714, 1.1)


In [11]:
import nibabel as nib

img = nib.load("images/DUKE_001_0001_n4_corrected_denoised_resampled.nii.gz")
voxel_size = img.header.get_zooms()[:3]
print("Resampled voxel size (mm):", voxel_size)


Resampled voxel size (mm): (0.80357134, 0.8035714, 1.1)


## Check for unprocessed samples

In [ ]:
import os
import pandas as pd

# ---------- Load CSV ----------
csv_path = "../train_test_splits.csv"
df = pd.read_csv(csv_path)

# Filter to only rows starting with "DUKE"
train_ids = [pid for pid in df["train_split"].dropna() if str(pid).startswith("DUKE")]
test_ids = [pid for pid in df["test_split"].dropna() if str(pid).startswith("DUKE")]

expected_ids = {
    "train_0001": train_ids,
    "test_0001": test_ids
}

# ---------- Check which ones are missing ----------
missing_patients = []

for folder, expected_list in expected_ids.items():
    loaded_ids = []
    for pid in expected_list:
        patient_folder = os.path.join(folder, pid)
        expected_filename = f"{pid}_0001_n4_corrected_denoised_resampled.nii.gz"
        full_path = os.path.join(patient_folder, expected_filename)
        if os.path.exists(full_path):
            loaded_ids.append(pid)
    missing = sorted(set(expected_list) - set(loaded_ids))
    if missing:
        print(f"[{folder}] Missing {len(missing)} patients:")
        for m in missing:
            print(f"  - {m}")
        missing_patients.extend(missing)

print(f"\nTotal missing: {len(missing_patients)}")
# ---------- Save missing patient IDs to text file ----------
output_path = "missing_patients.txt"
with open(output_path, "w") as f:
    for pid in missing_patients:
        f.write(f"{pid}\n")

print(f"\n✅ Saved missing patient IDs to: {output_path}")


## Get sizes of images

In [ ]:
import os
import nibabel as nib
import numpy as np

base_dirs = ["train_0001", "test_0001"]
image_shapes = []

for base_dir in base_dirs:
    print(f"[INFO] Scanning directory: {base_dir}")
    for patient_id in os.listdir(base_dir):
        patient_path = os.path.join(base_dir, patient_id)
        if not os.path.isdir(patient_path):
            continue
        for file in os.listdir(patient_path):
            if file.endswith("_0001_n4_corrected_denoised_resampled.nii.gz"):
                file_path = os.path.join(patient_path, file)
                try:
                    img = nib.load(file_path)
                    shape = img.shape
                    image_shapes.append(shape)
                    print(f"{file_path} => shape: {shape}")
                except Exception as e:
                    print(f"[ERROR] Failed to load {file_path}: {e}")

# Convert to NumPy array for analysis
shape_array = np.array(image_shapes)
avg_shape = np.mean(shape_array, axis=0)
std_shape = np.std(shape_array, axis=0)

print("\n========== Summary ==========")
print(f"Number of images: {len(image_shapes)}")
print(f"Average shape: {avg_shape}")
print(f"Standard deviation: {std_shape}")


## Get Size of 1 Image

In [27]:
import nibabel as nib

# --- Set your file path here ---
file_path = r"C:\Users\edwin\REU 2025\images\DUKE_002\duke_002_0001.nii"

# --- Load the image ---
img = nib.load(file_path)
img_data = img.get_fdata()

# --- Print the shape (size in voxels) ---
print(f"Image shape (z, y, x): {img_data.shape}")


FileNotFoundError: No such file or no access: 'C:/Users/edwin/REU 2025/images/DUKE_002/duke_002_0001.nii'

## Print Mask + Image Size

In [1]:
import nibabel as nib

image_path = "images/DUKE_001/duke_001_0001.nii.gz"
mask_path = "segmentations/automatic/DUKE_001.nii.gz"

# Load image
img = nib.load(image_path)
print("[Image]")
print("  Shape:", img.shape)
print("  Spacing:", img.header.get_zooms()[:3])

# Load mask
mask = nib.load(mask_path)
print("[Mask]")
print("  Shape:", mask.shape)
print("  Spacing:", mask.header.get_zooms()[:3])


[Image]
  Shape: (448, 448, 160)
  Spacing: (0.8035714, 0.8035714, 1.1)
[Mask]
  Shape: (448, 448, 160)
  Spacing: (0.8035714, 0.8035714, 1.1)


## Check Range of Mask

In [14]:
import nibabel as nib
import numpy as np

# Path to your mask
mask_path = "segmentations/automatic/DUKE_001_processed.nii.gz"

# Load mask
mask = nib.load(mask_path).get_fdata()

# Get unique values
unique_values = np.unique(mask)
print(f"Unique values in mask: {unique_values}")

# Check if it's binary
is_binary = np.array_equal(unique_values, [0, 1])
print("Is binary mask:", is_binary)

Unique values in mask: [0. 1.]
Is binary mask: True


## Count Number of Images

In [17]:
import os

# Path to your images directory
images_dir = "images"

# List all entries in the directory
all_entries = os.listdir(images_dir)

# Filter for directories starting with "DUKE"
duke_folders = [
    name for name in all_entries
    if name.startswith("DUKE") and os.path.isdir(os.path.join(images_dir, name))
]

# Count and print the number
print(f"Number of folders starting with 'DUKE': {len(duke_folders)}")


Number of folders starting with 'DUKE': 291


## Flip One Image Along Axial Plane

In [24]:
import nibabel as nib
import numpy as np
import os

# ---- Input Paths ----
image_path = "images/DUKE_001_0001_n4_denoised_resampled_padded.nii.gz"
mask_path = "segmentations/automatic/DUKE_001_processed.nii.gz"

# ---- Output Paths ----
flipped_image_path = image_path.replace(".nii.gz", "_flipped_LR.nii.gz")
flipped_mask_path = mask_path.replace(".nii.gz", "_flipped_LR.nii.gz")

def flip_lr(input_path, output_path):
    print(f"[INFO] Flipping: {input_path}")
    nib_img = nib.load(input_path)
    data = nib_img.get_fdata()
    flipped_data = np.flip(data, axis=0)  # Flip along x-axis (left-right)
    flipped_nib = nib.Nifti1Image(flipped_data, nib_img.affine)
    nib.save(flipped_nib, output_path)
    print(f"[✅ SAVED] Flipped output: {output_path}")

# ---- Flip both image and mask ----
flip_lr(image_path, flipped_image_path)
flip_lr(mask_path, flipped_mask_path)


[INFO] Flipping: images/DUKE_001_0001_n4_denoised_resampled_padded.nii.gz
[✅ SAVED] Flipped output: images/DUKE_001_0001_n4_denoised_resampled_padded_flipped_LR.nii.gz
[INFO] Flipping: segmentations/automatic/DUKE_001_processed.nii.gz
[✅ SAVED] Flipped output: segmentations/automatic/DUKE_001_processed_flipped_LR.nii.gz


## Flip All Images

In [ ]:
import os
import nibabel as nib
import numpy as np

# ---- Directories ----
images_root = "images"
masks_root = "segmentations/automatic"

# ---- Utility Function ----
def flip_lr(input_path, output_path):
    nib_img = nib.load(input_path)
    data = nib_img.get_fdata()
    flipped_data = np.flip(data, axis=0)  # Flip along x-axis (left-right)
    flipped_nib = nib.Nifti1Image(flipped_data, nib_img.affine)
    nib.save(flipped_nib, output_path)
    print(f"[✅ SAVED] Flipped: {output_path}")

# ---- Main Loop ----
all_patient_ids = sorted(os.listdir(images_root))

for patient_id in all_patient_ids:
    if not patient_id.startswith("DUKE"):
        continue

    # Construct paths
    image_path = os.path.join(images_root, patient_id, f"{patient_id}_0001_n4_denoised_resampled_padded.nii.gz")
    mask_path = os.path.join(masks_root, f"{patient_id}_processed.nii.gz")

    # Check existence
    if not os.path.exists(image_path):
        print(f"[SKIP] Image not found: {image_path}")
        continue

    if not os.path.exists(mask_path):
        print(f"[SKIP] Mask not found: {mask_path}")
        continue

    # Output paths
    flipped_image_path = image_path.replace(".nii.gz", "_flipped_LR.nii.gz")
    flipped_mask_path = mask_path.replace(".nii.gz", "_flipped_LR.nii.gz")

    print(f"\n[PROCESSING] {patient_id}")
    flip_lr(image_path, flipped_image_path)
    flip_lr(mask_path, flipped_mask_path)


## Count Classification Proportion

In [ ]:
import os
import pandas as pd

# Load quality scores
quality_path = "../preliminary_automatic_segmentations_quality_scores.csv"
quality_df = pd.read_csv(quality_path)

# Identify all flipped image-mask pairs
image_root = "../images"
mask_root = "../segmentations/automatic"

valid_pairs = []
good_pairs = []

# Create set of "both good" patient_ids
good_ids = set(
    quality_df[
        (quality_df["expert_1_qs"] == "Good") &
        (quality_df["expert_2_qs"] == "Good")
    ]["patient_id"]
)

# Traverse through the images directory
for patient_folder in os.listdir(image_root):
    patient_id = patient_folder
    image_path = os.path.join(image_root, patient_id, f"{patient_id}_0001_n4_denoised_resampled_padded_flipped_LR.nii.gz")
    mask_path = os.path.join(mask_root, f"{patient_id}_processed_flipped_LR.nii.gz")

    if os.path.isfile(image_path) and os.path.isfile(mask_path):
        valid_pairs.append(patient_id)
        if patient_id in good_ids:
            good_pairs.append(patient_id)

# Output results
print(f"Total valid image-mask pairs: {len(valid_pairs)}")
print(f"Valid pairs with both expert scores 'Good': {len(good_pairs)}")
print(f"Proportion: {len(good_pairs) / len(valid_pairs):.4f}")


## Last Processed Index

In [7]:
# Load and parse the uploaded file to identify the highest index mentioned
file_path = "preprocess_12184073.out"

with open(file_path, "r") as file:
    lines = file.readlines()

# Extract all lines containing index information
import re
index_patient_pairs = []
for line in lines:
    match = re.search(r"\[INDEX\] Processing patient index (\d+) \((DUKE_\d+)\)", line)
    if match:
        index = int(match.group(1))
        patient_id = match.group(2)
        index_patient_pairs.append((index, patient_id))

# Get the pair with the highest index
highest_pair = max(index_patient_pairs, key=lambda x: x[0]) if index_patient_pairs else None
highest_pair


(214, 'DUKE_656')

### N4 Test

In [ ]:
import os
import nibabel as nib
import numpy as np
import SimpleITK as sitk

# ---- Configuration ----
patient_id = "DUKE_001"
images_root = "../images"
preprocessed_root = "../preprocessed"
output_dir = os.path.join(preprocessed_root, patient_id, "n4_test")
os.makedirs(output_dir, exist_ok=True)

def n4_bias_correction_with_metadata(nib_image):
    print("[STEP] Running N4 bias correction...")

    # Convert nibabel image to SimpleITK image
    image_np = nib_image.get_fdata().astype(np.float32)
    sitk_image = sitk.GetImageFromArray(image_np)

    # Extract spatial metadata from nibabel
    zooms = tuple(float(z) for z in nib_image.header.get_zooms()[:3])
    sitk_image.SetSpacing(zooms)
    sitk_image.SetOrigin((0.0, 0.0, 0.0))  # origin is arbitrary if not stored; safe default
    sitk_image.SetDirection((1.0, 0.0, 0.0,
                             0.0, 1.0, 0.0,
                             0.0, 0.0, 1.0))  # assume axial acquisition

    mask_image = sitk.Cast(sitk_image > 0, sitk.sitkUInt8)
    corrector = sitk.N4BiasFieldCorrectionImageFilter()
    corrected = corrector.Execute(sitk_image, mask_image)

    print("[DONE] N4 bias correction complete.")
    return sitk.GetArrayFromImage(corrected)


# ---- Process DUKE_001 ----
patient_folder = os.path.join(images_root, patient_id)
for fname in os.listdir(patient_folder):
    if fname.startswith("DUKE_") and fname.endswith("0001.nii.gz"):
        input_path = os.path.join(patient_folder, fname)
        print(f"[PROCESSING] {input_path}", flush=True)

        # Load and correct
        img_nib = nib.load(input_path)
        img_n4 = n4_bias_correction_with_metadata(img_nib)


        # Save output
        output_path = os.path.join(output_dir, "n4.nii.gz")
        nib.save(nib.Nifti1Image(img_n4, img_nib.affine), output_path)
        print(f"[✅ SAVED] N4 corrected image to {output_path}")
        break  # Only process one image
